In [1]:
from fpl import Loader, Player, Team, ExpectedPointsCalculator, Optimizer
from typing import List
import heapq

### Dowloading the team

In [2]:
picks = Loader.get_my_team_from_api("aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546)['picks']
gkps, defs, mids, fwds = set(), set(), set(), set()
for pick in picks:
    d = {'element': pick['element'],
         'name': Loader.get_player_basic_info(pick['element'])['web_name'],
         'position': Loader.get_player_basic_info(pick['element'])['element_type'],
         'club': Loader.get_player_basic_info(pick['element'])['team'],
         'cost': pick['selling_price'],
        }
    player = Player(**d)
    if player.position == 1: gkps.add(player)
    if player.position == 2: defs.add(player)
    if player.position == 3: mids.add(player)
    if player.position == 4: fwds.add(player)

money_in_bank = Loader.get_my_team_from_api("aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546)['transfers']['bank']
free_transfers = Loader.get_my_team_from_api("aledvaghela@gmail.com", "NCP@jxy7pau-bvm3cuf", 3247546)['transfers']['limit']
free_transfers = 0 if free_transfers is None else free_transfers
team = Team(money_in_bank, free_transfers, frozenset(gkps), frozenset(defs), frozenset(mids), frozenset(fwds))
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=109, name='Valdimarsson', position=1, club=4, cost=40)
Player(element=413, name='Pope', position=1, club=15, cost=50)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=60)
Player(element=116, name='Barco', position=2, club=5, cost=40)
Player(element=211, name='Muñoz', position=2, club=7, cost=50)
Player(element=231, name='Mykolenko', position=2, club=8, cost=45)
Player(element=350, name='Gvardiol', position=2, club=13, cost=60)
---------------------------------------------------------------------------
{MIDS}
Player(element=17, name='Saka', position=3, club=1, cost=100)
Player(element=181, name='Nkunku', position=3, club=6, cost=65)
Player(element=182, name='Palmer', position=3, club=6, cost=105)
Player(element=199, name='Eze', position=3, club=7, cost=70)
Player(element=328, name='M.Salah', positio

### Find a player

In [3]:
search_name = "ré"
for i in [p["id"] for p in Loader.get_static_info()["elements"] if p["web_name"].endswith(search_name)]:
    print(f"id: {i}, first name: {Loader.get_player_basic_info(i)["first_name"]}, second name: {Loader.get_player_basic_info(i)["second_name"]}")

id: 193, first name: Cheick, second name: Doucouré
id: 285, first name: Boubakary, second name: Soumaré
id: 442, first name: Ibrahim, second name: Sangaré


In [4]:
Loader.get_player_basic_info(54)

{'chance_of_playing_next_round': None,
 'chance_of_playing_this_round': None,
 'code': 244850,
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': 0,
 'cost_change_start_fall': 0,
 'dreamteam_count': 0,
 'element_type': 3,
 'ep_next': '1.5',
 'ep_this': None,
 'event_points': 0,
 'first_name': 'Morgan',
 'form': '0.0',
 'id': 54,
 'in_dreamteam': False,
 'news': '',
 'news_added': None,
 'now_cost': 50,
 'photo': '244850.jpg',
 'points_per_game': '3.2',
 'second_name': 'Rogers',
 'selected_by_percent': '8.2',
 'special': False,
 'squad_number': None,
 'status': 'a',
 'team': 2,
 'team_code': 7,
 'total_points': 35,
 'transfers_in': 0,
 'transfers_in_event': 0,
 'transfers_out': 0,
 'transfers_out_event': 0,
 'value_form': '0.0',
 'value_season': '7.0',
 'web_name': 'Rogers',
 'minutes': 636,
 'goals_scored': 3,
 'assists': 1,
 'clean_sheets': 2,
 'goals_conceded': 13,
 'own_goals': 0,
 'penalties_saved': 0,
 'penalties_missed': 0,
 'yellow_cards': 3,
 'red_card

### Candidate list

In [5]:
candidates_gkps = [15, 310, 413, 91, 201, 109]
candidates_defs = [120, 311, 18, 339, 350, 495, 211, 44, 255, 231, 275, 116, 461]
candidates_mids = [503, 17, 328, 19, 442, 54, 78, 434, 433, 28, 181, 372, 491, 398, 199, 13, 366, 317, 182]
candidates_fwds = [351, 58, 401, 447, 251, 286, 82, 4]

### How to calculate points

In [6]:
class ComplexExpectedPointsCalculator(ExpectedPointsCalculator):
    minutes_multiplier = {
        
        116: 90/90, # barco
        231: 90/90, # mykolenko
        461: 90/90, # bellis
        275: 90/90, # ben johnson
        120: 90/90, # dunk
        311: 90/90, # arnold
        18: 90/90, # saliba
        339: 90/90, # virgil
        350: 90/90, # gvardiol
        495: 90/90, # porro
        211: 90/90, # munoz
        44: 90/90, # konsa
        255: 90/90, # robinson
        
        181: 75/90, # nkunku
        78: 80/90, # semenyo
        434: 67.5/90, # hudson odoi
        491: 67.5/90, # brennan johnson
        372: 67.5/90, # garnacho
        28: 75/90, # bailey
        54: 65/90, # rogers
        503: 85/90, # son
        182: 85/90, # palmer
        17: 85/90, # saka
        328: 85/90, # salah
        199: 80/90, # eze
        19: 80/90, # smith rowe
        442: 85/90, # sangare
        433: 85/90, # gibbs white
        398: 67.5/90, # gordon
        13: 87.5/90, # odergard
        366: 90/90, # fernandes
        317: 65/90, # jota
        
        401: 80/90, # isak
        58: 60/90, # watkins
        251: 85/90, # muniz
        447: 65/90, # wood - awonyi / other striker may displace
        351: 80/90, # haaland
        286: 50/90, # cannon
        82: 70/90, # solanke
        4: 80/90, # havertz
    }
    points_per_game_override = {
        116: 3.0, # barco
        461: 2.0, # bellis
        275: 2.0, # ben johnson
        
        181: 5.0, # nkunku
        19: 3.25, # smith rowe
        286: 2.0, # cannon
        
        82: 5.0, # solanke
        4: 4.5, # havertz
    }
    def get_expected_points(player_id: int, gameweek: int) -> float:
        points_per_game = 0
        if player_id in ComplexExpectedPointsCalculator.points_per_game_override:
            points_per_game = ComplexExpectedPointsCalculator.points_per_game_override[player_id]
        else:
            points_per_game = float(Loader.get_player_basic_info(player_id)['points_per_game'])
        
        result = 0
        for fixture in Loader.get_player_future_info_for_gameweek(player_id, gameweek):
            our_team, opponent = -1, -1
            if fixture['is_home']:
                our_team = fixture['team_h']
                opponent = fixture['team_a']
            else:
                our_team = fixture['team_a']
                opponent = fixture['team_h']
            
            difficulty_alpha = 0
            if fixture['difficulty'] == 2:
                difficulty_alpha = 1.5
            elif fixture['difficulty'] == 3:
                difficulty_alpha = 0.75
            elif fixture["difficulty"] == 4:
                difficulty_alpha = -0.75
            elif fixture["difficulty"] == 5:
                difficulty_alpha = -1.5
            else:
                raise ValueError("Fixture difficulty should be be 2,3,4,5")
            
            result += points_per_game + difficulty_alpha
        
        # because of the new change to the bonus points system
        if Loader.get_player_basic_info(player_id)['element_type'] in [1,2]:
            result -= 0.75
        
        # for goalies assume they start and play 90
        if Loader.get_player_basic_info(player_id)['element_type'] == 1:
            # manually set valdimarson 4.0m gkp pts to zero
            return result if player_id != 109 else 0
        
        if player_id not in ComplexExpectedPointsCalculator.minutes_multiplier:
            web_name = Loader.get_player_basic_info(player_id)['web_name']
            raise KeyError(f"You need to get the minutes for {player_id} {web_name}")
            
        
        return result * ComplexExpectedPointsCalculator.minutes_multiplier[player_id]

In [7]:
def get_discounted_reward(player_id: int, number_gameweeks: int, gamma: float = 1) -> float:
    """Gets the discounted reward of a candidate over a certain number of gameweeks
    :param player_id: the player which you want the reward for
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor
    
    :return: discounted reward
    """
    discounted_reward = 0
    discount_factor = 1
    for gw in range(1, number_gameweeks+1):
        discounted_reward += discount_factor * ComplexExpectedPointsCalculator.get_expected_points(player_id, gw)
        discount_factor *= gamma
    
#     return discounted_reward
    return discounted_reward / Loader.get_player_basic_info(player_id)["now_cost"]

def print_sorted_candidates(candidates: List[int], number_gameweeks: int, gamma: float = 1) -> None:
    """Print out the list of players and rewards they get
    :param candidates: list of player ids
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor
    """
    sorted_candidates = []
    for i in candidates:
        r = get_discounted_reward(i, number_gameweeks, gamma)
        sorted_candidates.append( (i, r) )

    sorted_candidates = reversed(sorted(sorted_candidates, key=lambda x: x[1]))
    for i, r in sorted_candidates:
        print(f"id: {i}, name: {Loader.get_player_basic_info(i)["web_name"]}, reward: {r}")
    
    return

In [8]:
print_sorted_candidates(candidates_gkps+candidates_defs+candidates_mids+candidates_fwds, 8, 0.80)

id: 413, name: Pope, reward: 0.3393267072
id: 231, name: Mykolenko, reward: 0.33227438933333336
id: 310, name: A.Becker, reward: 0.3290779810909091
id: 199, name: Eze, reward: 0.3106618514285715
id: 251, name: Muniz, reward: 0.30910101155555564
id: 211, name: Muñoz, reward: 0.30204115200000003
id: 181, name: Nkunku, reward: 0.3002340923076924
id: 15, name: Raya, reward: 0.2984879825454546
id: 311, name: Alexander-Arnold, reward: 0.29422817828571435
id: 495, name: Pedro Porro, reward: 0.2932279505454546
id: 182, name: Palmer, reward: 0.2929825462857143
id: 401, name: Isak, reward: 0.2927409593725491
id: 433, name: Gibbs-White, reward: 0.2895552426666667
id: 19, name: Smith Rowe, reward: 0.28710291393939397
id: 350, name: Gvardiol, reward: 0.2868964480000001
id: 255, name: Robinson, reward: 0.2838027946666667
id: 18, name: Saliba, reward: 0.280549216
id: 13, name: Ødegaard, reward: 0.2758245521568628
id: 17, name: Saka, reward: 0.2749118506666667
id: 116, name: Barco, reward: 0.273213120

### Optimizing the team

In [9]:
candidates = []
for pid in candidates_gkps + candidates_defs + candidates_mids + candidates_fwds:
    d = {'element': pid,
         'name': Loader.get_player_basic_info(pid)['web_name'],
         'position': Loader.get_player_basic_info(pid)['element_type'],
         'club': Loader.get_player_basic_info(pid)['team'],
         'cost': Loader.get_player_basic_info(pid)['now_cost'],
        }
    player = Player(**d)
    if player in gkps | defs | mids | fwds:
        print(f"Already have: {player}")
        continue
    else:
        candidates.append(player)

Already have: Player(element=413, name='Pope', position=1, club=15, cost=50)
Already have: Player(element=109, name='Valdimarsson', position=1, club=4, cost=40)
Already have: Player(element=18, name='Saliba', position=2, club=1, cost=60)
Already have: Player(element=350, name='Gvardiol', position=2, club=13, cost=60)
Already have: Player(element=211, name='Muñoz', position=2, club=7, cost=50)
Already have: Player(element=231, name='Mykolenko', position=2, club=8, cost=45)
Already have: Player(element=116, name='Barco', position=2, club=5, cost=40)
Already have: Player(element=17, name='Saka', position=3, club=1, cost=100)
Already have: Player(element=328, name='M.Salah', position=3, club=12, cost=125)
Already have: Player(element=181, name='Nkunku', position=3, club=6, cost=65)
Already have: Player(element=199, name='Eze', position=3, club=7, cost=70)
Already have: Player(element=182, name='Palmer', position=3, club=6, cost=105)
Already have: Player(element=401, name='Isak', position=4

In [10]:
top_three = Optimizer.optimize_team(team, 
                                    candidates, 
                                    ComplexExpectedPointsCalculator, 
                                    gameweek=1, 
                                    horizon=8, 
                                    max_transfers=3,
                                    gamma=0.8,
                                    wildcard=True
                                   )
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)

In [17]:
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 281.5127947733333
---------------------------------------------------------------------------
{GKPS}
Player(element=109, name='Valdimarsson', position=1, club=4, cost=40)
Player(element=413, name='Pope', position=1, club=15, cost=50)
---------------------------------------------------------------------------
{DEFS}
Player(element=211, name='Muñoz', position=2, club=7, cost=50)
Player(element=231, name='Mykolenko', position=2, club=8, cost=45)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=70)
Player(element=350, name='Gvardiol', position=2, club=13, cost=60)
Player(element=495, name='Pedro Porro', position=2, club=18, cost=55)
---------------------------------------------------------------------------
{MIDS}
Player(element=17, name='Saka', position=3, club=1, cost=100)
Player(element=181, name='Nkunku', position=3, club=6, cost=65)
Player(element=182, name='Palmer', position=3, club=6, cost=105)
Player(element=199, name='Eze', position=3, club=7, cost=70)

In [18]:
print("Score: {}".format(second_best_score))
print(second_best_team)

Score: 281.2210030577778
---------------------------------------------------------------------------
{GKPS}
Player(element=109, name='Valdimarsson', position=1, club=4, cost=40)
Player(element=310, name='A.Becker', position=1, club=12, cost=55)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=60)
Player(element=116, name='Barco', position=2, club=5, cost=40)
Player(element=231, name='Mykolenko', position=2, club=8, cost=45)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=70)
Player(element=350, name='Gvardiol', position=2, club=13, cost=60)
---------------------------------------------------------------------------
{MIDS}
Player(element=17, name='Saka', position=3, club=1, cost=100)
Player(element=181, name='Nkunku', position=3, club=6, cost=65)
Player(element=182, name='Palmer', position=3, club=6, cost=105)
Player(element=199, name='Eze', position=3, club=7, cost=70)
Pl

In [35]:
print("Score: {}".format(third_best_score))
print(third_best_team)

Score: 281.16213763555555
---------------------------------------------------------------------------
{GKPS}
Player(element=15, name='Raya', position=1, club=1, cost=55)
Player(element=413, name='Pope', position=1, club=15, cost=50)
---------------------------------------------------------------------------
{DEFS}
Player(element=18, name='Saliba', position=2, club=1, cost=60)
Player(element=116, name='Barco', position=2, club=5, cost=40)
Player(element=211, name='Muñoz', position=2, club=7, cost=50)
Player(element=231, name='Mykolenko', position=2, club=8, cost=45)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=70)
---------------------------------------------------------------------------
{MIDS}
Player(element=17, name='Saka', position=3, club=1, cost=100)
Player(element=181, name='Nkunku', position=3, club=6, cost=65)
Player(element=182, name='Palmer', position=3, club=6, cost=105)
Player(element=199, name='Eze', position=3, club=7, cost=70)
Player(element=503

In [34]:
Optimizer.optimal_formation(first_best_team, ComplexExpectedPointsCalculator, 7)

{'gkps': frozenset({Player(element=413, name='Pope', position=1, club=15, cost=50)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=45),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=70),
            Player(element=350, name='Gvardiol', position=2, club=13, cost=60),
            Player(element=495, name='Pedro Porro', position=2, club=18, cost=55)}),
 'mids': frozenset({Player(element=17, name='Saka', position=3, club=1, cost=100),
            Player(element=181, name='Nkunku', position=3, club=6, cost=65),
            Player(element=182, name='Palmer', position=3, club=6, cost=105),
            Player(element=199, name='Eze', position=3, club=7, cost=70),
            Player(element=503, name='Son', position=3, club=18, cost=100)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=85)}),
 'captain': Player(element=182, name='Palmer', position=3, club=6, cost=105),
 'total_exp_points'

In [42]:
Optimizer.optimal_formation(second_best_team, ComplexExpectedPointsCalculator, 7)

{'gkps': frozenset({Player(element=310, name='A.Becker', position=1, club=12, cost=55)}),
 'defs': frozenset({Player(element=18, name='Saliba', position=2, club=1, cost=60),
            Player(element=231, name='Mykolenko', position=2, club=8, cost=45),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=70),
            Player(element=350, name='Gvardiol', position=2, club=13, cost=60)}),
 'mids': frozenset({Player(element=17, name='Saka', position=3, club=1, cost=100),
            Player(element=181, name='Nkunku', position=3, club=6, cost=65),
            Player(element=182, name='Palmer', position=3, club=6, cost=105),
            Player(element=199, name='Eze', position=3, club=7, cost=70),
            Player(element=503, name='Son', position=3, club=18, cost=100)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=85)}),
 'captain': Player(element=182, name='Palmer', position=3, club=6, cost=105),
 'total_exp_points': 6

In [16]:
Optimizer.optimal_formation(team, ComplexExpectedPointsCalculator, 4)

{'gkps': frozenset({Player(element=413, name='Pope', position=1, club=15, cost=50)}),
 'defs': frozenset({Player(element=116, name='Barco', position=2, club=5, cost=40),
            Player(element=211, name='Muñoz', position=2, club=7, cost=50),
            Player(element=350, name='Gvardiol', position=2, club=13, cost=60)}),
 'mids': frozenset({Player(element=17, name='Saka', position=3, club=1, cost=100),
            Player(element=181, name='Nkunku', position=3, club=6, cost=65),
            Player(element=182, name='Palmer', position=3, club=6, cost=105),
            Player(element=199, name='Eze', position=3, club=7, cost=70),
            Player(element=328, name='M.Salah', position=3, club=12, cost=125)}),
 'fwds': frozenset({Player(element=251, name='Muniz', position=4, club=9, cost=60),
            Player(element=401, name='Isak', position=4, club=15, cost=85)}),
 'captain': Player(element=182, name='Palmer', position=3, club=6, cost=105),
 'total_exp_points': 69.71944444444445